# GDC to CCDH Conversion

This notebook demonstrates one method for converting GDC data into CCDH (CRDC-H) instance data: by reading node data as JSON and writing it out in the LinkML model. The LinkML can be used to [generate](https://github.com/linkml/linkml#python-dataclasses) [Python Data Classes](https://docs.python.org/3/library/dataclasses.html), which can then be exported in JSON-LD, a JSON-based format used to represent RDF data.

## Why Python Data Classes?

Python Data Classes provide several useful features that we will demonstrate below:

1. **Python Data Classes are generated automatically.** Rather than requiring additional effort to maintain a Python library for accessing the CCDH model, the [LinkML toolset](https://linkml.github.io/) can generate the Python Data Classes directly from the CCDH model, ensuring that users can always access the most recent version of the CCDH model programmatically. This also allows us to maintain Python Data Classes for accessing previous versions of the CCDH model, which we plan to use to implement [data migration between CCDH model versions](https://cancerdhc.github.io/ccdhmodel/latest/data-migration/)
2. **Python Data Classes provide validation on creation.** As we will demonstrate below, creating a Python Data Class requires that all required attributes are filled in, and all fields are filled in the format or enumeration expected.
3. **Easy to use in Python IDEs.** Since the generated Python Data Classes includes model documentation in Python, users using Python IDEs can see available options and documentation while writing their code.

## Setup

We start by installing the [LinkML](https://pypi.org/project/linkml/) and [pandas](https://pypi.org/project/pandas/) packages. This is included in the pipenv file included in this source repository: if you used `pipenv run jupyter notebook` to start this Notebook, you should be set up already. If not, you may need to uncomment these following lines to install these packages.

(Note: if you are running this on macOS 11 "Big Sur", you might need to set `SYSTEM_VERSION_COMPAT=1` in your Terminal environment before running `pipenv install`).

In [1]:
import sys

# Install LinkML.
# We use our own fork of LinkML, but all changes made to this repository will eventually be sent
# upstream to the main LinkML release.
#!{sys.executable} -m pip install linkml

# Install pandas.
#!{sys.executable} -m pip install pandas

# Install rdflib.
#!{sys.executable} -m pip install rdflib

# Install JSON Schema.
#!{sys.executable} -m pip install jsonschema

## Loading GDC data as an example

In this demonstration, we will use a dataset of 560 cases relating to head and neck cancers previously downloaded from the public GDC API as [documented elsewhere in this repository](https://github.com/cancerDHC/example-data/blob/main/head-and-mouth/Head%20and%20Mouth%20Cancer%20Datasets.ipynb).

In [2]:
import json
import pandas

with open('head-and-mouth/gdc-head-and-mouth.json') as file:
    gdc_head_and_mouth = json.load(file)
    
pandas.DataFrame(gdc_head_and_mouth)

,aliquot_ids,case_id,created_datetime,diagnoses,diagnosis_ids,disease_type,id,primary_site,sample_ids,samples,...,submitter_sample_ids,submitter_slide_ids,updated_datetime,analyte_ids,portion_ids,submitter_analyte_ids,submitter_portion_ids,days_to_lost_to_followup,index_date,lost_to_followup
0,[cfcde639-3045-4f66-84a6-ec74b090a5b6],cd7e514f-71ba-4cc1-b74a-a22c6248169c,2017-06-01T08:57:57.249456-05:00,"[{'age_at_diagnosis': 19592, 'classification_o...",[5d2d67d1-4611-4a18-9a66-89823aaa8e3c],Adenomas and Adenocarcinomas,cd7e514f-71ba-4cc1-b74a-a22c6248169c,Nasopharynx,[bdc73f48-dc0b-487d-abbe-e3a977b6830a],[{'created_datetime': '2017-06-01T10:44:57.790...,...,[AD6426_sample],[AD6426_slide],2018-10-25T11:34:27.425461-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[9069bdd7-e16a-462c-881c-581c8aab6910, a74915f...",9023c9bf-02a0-4396-8161-304089957b62,None,"[{'age_at_diagnosis': 24286, 'ajcc_clinical_m'...",[706b1290-3a85-54ea-a123-e8bd14b085bc],Squamous Cell Neoplasms,9023c9bf-02a0-4396-8161-304089957b62,Larynx,"[8b2588c8-4261-492b-b173-2490a5de668f, badeaed...",[{'created_datetime': '2018-05-17T12:19:46.292...,...,"[TCGA-CN-6012-10A, TCGA-CN-6012-01A, TCGA-CN-6...","[TCGA-CN-6012-01Z-00-DX1, TCGA-CN-6012-01A-01-...",2019-08-06T14:25:25.511101-05:00,"[80c6fde2-b6bb-4f40-908a-f116c466d296, 6f77017...","[bada788e-5112-4d21-a079-72729bd0cc83, fe24eea...","[TCGA-CN-6012-01A-11D, TCGA-CN-6012-10A-01W, T...","[TCGA-CN-6012-01A-13-2072-20, TCGA-CN-6012-10A...",NaN,NaN,NaN
2,"[8f695cd3-01dd-4601-8b17-37cf40514422, f0e325f...",55f96a9c-e2c8-4243-8a7e-94bc6fab73a6,None,"[{'age_at_diagnosis': 20992, 'ajcc_clinical_m'...",[40954a8e-e4c2-5604-937b-0a79ac7489d2],Squamous Cell Neoplasms,55f96a9c-e2c8-4243-8a7e-94bc6fab73a6,Larynx,"[a7692585-a129-4671-bfe5-98342a326776, b069c55...","[{'composition': None, 'created_datetime': Non...",...,"[TCGA-CV-7261-01Z, TCGA-CV-7261-11A, TCGA-CV-7...","[TCGA-CV-7261-01A-01-TS1, TCGA-CV-7261-01Z-00-...",2019-08-06T14:26:28.608672-05:00,"[a72f2de7-eb40-4818-a104-edb508d5517b, e8120e5...","[177fa10b-0135-468d-b5a3-6f30cc3cd390, f51d76a...","[TCGA-CV-7261-10A-01D, TCGA-CV-7261-01A-11R, T...","[TCGA-CV-7261-10A-01, TCGA-CV-7261-01A-13-2074...",NaN,NaN,NaN
3,"[1265fd12-4706-43b0-84f3-d16d46f20963, 3443e1b...",c9a36eb5-ac3e-424e-bc2e-303de7105957,None,"[{'age_at_diagnosis': 21886, 'ajcc_clinical_m'...",[48e8dd81-ed4d-5c54-af66-84e86477d5c8],Squamous Cell Neoplasms,c9a36eb5-ac3e-424e-bc2e-303de7105957,Oropharynx,"[256469d0-5f36-4966-bf4f-3b4297e55f43, bd90f96...","[{'composition': None, 'created_datetime': Non...",...,"[TCGA-BA-A6DL-10A, TCGA-BA-A6DL-01Z, TCGA-BA-A...","[TCGA-BA-A6DL-01Z-00-DX1, TCGA-BA-A6DL-01A-02-...",2019-08-06T14:25:14.243346-05:00,"[ec4487c1-6976-4161-9236-5e6810ed31b7, ffd1e03...","[7f327ef6-4fe6-40c8-aac7-731e051177bb, 2a4b0be...","[TCGA-BA-A6DL-01A-21D, TCGA-BA-A6DL-01A-21R, T...","[TCGA-BA-A6DL-10A-01, TCGA-BA-A6DL-01A-11-A45L...",NaN,NaN,NaN
4,"[59b70846-64f0-489e-8ea5-84a347aedeb8, c8e46ce...",4cffea0b-90a7-4c86-a73f-bb8feca3ada7,None,"[{'age_at_diagnosis': 14190, 'ajcc_clinical_m'...",[1da5c51a-ee25-51a6-a4c2-27d8fdcbe24e],Squamous Cell Neoplasms,4cffea0b-90a7-4c86-a73f-bb8feca3ada7,Tonsil,"[1ed245de-fea4-42c9-9197-773bcd12d2a8, 665d4bf...",[{'created_datetime': '2018-05-17T12:19:46.292...,...,"[TCGA-CN-5365-01Z, TCGA-CN-5365-10A, TCGA-CN-5...","[TCGA-CN-5365-01Z-00-DX1, TCGA-CN-5365-01A-01-...",2019-08-06T14:25:25.511101-05:00,"[d46b5e9b-3652-45a1-a91d-46277aea3916, 35122dd...","[38c5a4c1-6d01-4885-ba35-0032e6b835b0, 516f802...","[TCGA-CN-5365-01A-01D, TCGA-CN-5365-01A-01W, T...","[TCGA-CN-5365-10A-01, TCGA-CN-5365-01A-21-2072...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555,"[1d3b16fd-f98b-45ef-a423-861975f098b6, 0eabe3e...",97640ef0-0259-4244-95ba-48d28c60b372,None,"[{'age_at_diagnosis': 19621, 'ajcc_clinical_m'...",[b725e6d2-92c0-5585-9de7-14bb623b472e],Squamous Cell Neoplasms,97640ef0-0259-4244-95ba-48d28c60b372,Larynx,"[fb06ae75-8516-4

## Loading the Python classes for the CCDH model

The Python DataClasses for the CCDH model as available at https://github.com/cancerDHC/ccdhmodel/. The Python DataClasses cannot be directly loaded from this GitHub repository yet, but we [plan to implement this functionality soon](https://github.com/cancerDHC/ccdhmodel/issues/40). For now, we have copied the file into this repository so we can import them here.

Note that the Python Data Classes includes documentation on entities and enumerations.

In [3]:
# from ccdh import ccdhmodel as ccdh
import crdch_model as ccdh

# Documentation for an entity.
print(f"Documentation for Specimen: {ccdh.Specimen.__doc__}")


# Documentation for an entity.
print(f"Documentation for BodySite: {ccdh.BodySite.__doc__}")


Documentation for Specimen: 
    Any material taken as a sample from a biological entity (living or dead), or from a physical object or the
    environment. Specimens are usually collected as an example of their kind, often for use in some investigation.
    
Documentation for BodySite: 
    A site in the body of an organism, typically described in terms of an anatomical location and optional qualifiers
    (e.g. left/right, upper/lower). But body sites as defined here may include 'non-anatomic' sites, such as the
    location of an implanted medical device.
    


In [4]:
# Documentation for an enumeration.
# print(f"Documentation for Specimen.specimen_type: {ccdh.EnumCCDHSpecimenSpecimenType.__doc__}")
print(f"Documentation for Specimen.specimen_type: {ccdh.EnumCRDCHSpecimenSpecimenType.__doc__}")

# List of permissible values for Specimen.specimen_type
# wiht move to codeable concepts,this may be of limited usefulness now
print("Permissible values in enumeration Specimen.specimen_type:")
# pvalues = [pv for key, pv in ccdh.EnumCCDHSpecimenSpecimenType.__dict__.items() if isinstance(pv, ccdh.PermissibleValue)]
pvalues = [pv for key, pv in ccdh.EnumCRDCHSpecimenSpecimenType.__dict__.items() if isinstance(pv, ccdh.PermissibleValue)]
for pv in pvalues:
    print(f' - Value "{pv.text}": {pv.description}')

Documentation for Specimen.specimen_type: 
    A high-level type of specimen, based on its derivation provenance (i.e. how far removed it is from the original
    sample extracted from a source).
    
Permissible values in enumeration Specimen.specimen_type:
 - Value "portion": A physical sub-part taken from an existing specimen.
 - Value "aliquot": A specimen that results from the division of some parent specimen into equal amounts for downstream analysis.
 - Value "analyte": A specimen generated through the extraction of a specified class of substance/chemical (e.g. DNA, RNA, protein) from a parent specimen, which is stored in solution as an analyte.
 - Value "slide": A specimen that is mounted on a slide or coverslip for microscopic analysis.
 - Value "Fresh Specimen": A specimen representing the material that was directly collected from a subject (i.e. not generated through portioning, aliquoting, or analyte extraction from an existing specimen).


## Transforming GDC cases into CCDH Research Subject

The primary transformation we will demonstrate here is transforming a [GDC case](https://docs.gdc.cancer.gov/Data_Dictionary/viewer/#?view=table-definition-view&id=case) into a [CCDH Research Subject](https://cancerdhc.github.io/ccdhmodel/latest/ResearchSubject/). To do this, we need to translate three additional components as well:
* Each GDC case includes a diagnosis, which we need to transform into a [CCDH Diagnosis](https://cancerdhc.github.io/ccdhmodel/latest/Diagnosis/).
* Each GDC diagnosis includes a description of the cancer stage (see properties named `ajcc_*` in [the GDC documentation](https://docs.gdc.cancer.gov/Data_Dictionary/viewer/#?view=table-definition-view&id=diagnosis)). We will translate this into a [CCDH Cancer Stage Observation Set](https://cancerdhc.github.io/ccdhmodel/latest/CancerStageObservationSet/).
* Each GDC case contains a hierarchy of samples, portions, analytes, aliquots and slides. For the purposes of this demonstration, we will focus on transforming only the top-level specimens into [CCDH Specimens](https://cancerdhc.github.io/ccdhmodel/latest/Specimen/), but the same method can be used to transform other parts of the hierarchy. We plan to [include that transformation](https://github.com/cancerDHC/example-data/issues/6) in this tutorial eventually. Note that in our model, specimens are associated with diagnoses rather than directly with Research Subjects.

The CCDH Python Data Classes help in writing these transformation methods by applying validation on the data and ensuring that constraints (such as the required fields) are met. We begin by defining a transformation for creating a [CCDH BodySite](https://cancerdhc.github.io/ccdhmodel/latest/BodySite/), which we also use to demonstrate the validation features available on CCDH Python Data Classes.

In [5]:
import requests
import pandas as pd
import difflib

# use terminology.ccdh.io to drive mapping strings to codable concepts
# paramterize the URL below
response = requests.get(
    "https://terminology.ccdh.io/models/CRDC-H/entities/BodySite/attributes/site"
)
resp_struct = response.json()
BodyPart_frame = pd.DataFrame(resp_struct["concept_references"])

# could also use
# https://terminology.ccdh.io/enumerations/CRDC-H.BodySite.site


def create_body_site(site_name):
    """Create a CCDH BodySite based on the name of a site in the human body."""

    # Accept 'None'.
    if site_name is None:
        return None
    #
    #     # Some body sites are not currently included in the CCDH model. We will need to translate these sites
    #     # into values that *are* included in the CCDH model.
    # #     site_mappings = {
    # #         'Larynx, NOS': ccdh.EnumCCDHBodySiteSite.Larynx
    # #     }
    #     site_mappings = {
    #         'Larynx, NOS': ccdh.CodeableConcept(coding=[ccdh.Coding(code="UBERON:0001737", system="http://purl.obolibrary.org/obo/uberon.owl", label="larynx")])
    #     }
    #
    #     # previously: Map values if needed. Otherwise, pass them through unmapped.
    #     # now: can't pass through text because the site has to be a codable concept.
    #     # should the mapping be done here in the application, via the CCDH terminology serivce?
    #     # mock up for now with some other lookup service?
    #     if site_name in site_mappings:
    #         return ccdh.BodySite(site=(site_mappings[site_name]))
    #     else:
    #         return None
    #
    # #     return ccdh.BodySite(site=site_name)
    
    # blindly get closest match
    closest_BodyPart_label = difflib.get_close_matches(
        site_name, list(BodyPart_frame["designation"])
    )[0]
    # send to logger or wrap as an optional debug mode
    print("input site:    " + site_name)
    print("closest match: " + closest_BodyPart_label)
    closest_BodyPart = BodyPart_frame.loc[
        BodyPart_frame["designation"] == closest_BodyPart_label
    ]
    closest_BodyPart = closest_BodyPart.to_dict(orient="records")[0]
    print(closest_BodyPart)
    # this doesn't really say what the "system" should be
    mapped = ccdh.BodySite(
        site=ccdh.CodeableConcept(
            coding=[
                ccdh.Coding(
                    code=closest_BodyPart["code"],
                    system="https://terminology.ccdh.io/models/CRDC-H/entities/BodySite/attributes/site",
                    label=closest_BodyPart["designation"]
                )
            ], text = site_name
        )
    )
    return mapped


In [6]:
# # Try to create a body site for a site name not currently included in the CCDH model.
# try:
#     created = create_body_site('Laryn') # Note misspelling.
# except ValueError as v:
#     print(f'Could not create BodySite: {v}')
# print(created)

In [7]:
# # Using a valid name generates no errors.
# create_body_site('Larynx')    

# # Using a mapped name generates no errors, as it is mapped to a valid name.
# create_body_site('Larynx, NOS')

In [8]:
# how is this still useful? why is it still in the model?
ccdh.EnumCRDCHBodySiteSite.Larynx

PermissibleValue(text='Larynx', description='Larynx', meaning=None, is_a=None, mixins=[], extensions={}, annotations={}, alt_descriptions={}, deprecated=None, todos=[], notes=[], comments=[], examples=[], in_subset=[], from_schema=None, imported_from=None, see_also=[], deprecated_element_has_exact_replacement=None, deprecated_element_has_possible_replacement=None)

In [9]:
created_body_site = create_body_site("Laryn")

input site:    Laryn
closest match: Larynx
{'code': 'C12420', 'uri': 'http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#C12420', 'designation': 'Larynx', 'definition': 'The cartilaginous structure of the respiratory tract between the pharynx and the trachea. It contains elastic vocal cords required for sound production.', 'reference': 'https://ncit.nci.nih.gov/ncitbrowser/pages/concept_details.jsf?dictionary=NCI%20Thesaurus&code=C12420'}


In [10]:
created_body_site

BodySite(site=CodeableConcept(coding=[Coding(code='C12420', system='https://terminology.ccdh.io/models/CRDC-H/entities/BodySite/attributes/site', label='Larynx', system_version=None, value_set=None, value_set_version=None, tag=[])], text='Laryn'), qualifier=[])

---

We need a more sophisticated transformation method for transforming the GDC cancer stage information into [CCDH Cancer Stage Observation Set](https://cancerdhc.github.io/ccdhmodel/latest/CancerStageObservationSet/). Each observation set is made up of a number of [CCDH Cancer Stage Observations](https://cancerdhc.github.io/ccdhmodel/latest/CancerStageObservation/), each of which represents a different type of observation.

https://terminology.ccdh.io/models/CRDC-H/entities/CancerStageObservation/attributes

```
[
  {
    "name": "category"
  },
  {
    "name": "observation_type"
  },
  {
    "name": "method_type"
  },
  {
    "name": "valueCodeableConcept"
  },
  {
    "name": "id"
  },
  {
    "name": "focus"
  },
  {
    "name": "subject"
  },
  {
    "name": "performed_by"
  },
  {
    "name": "valueEntity"
  }
]
```

`enum_CRDCH_CancerStageObservation_observation_type` still has `permissible_values` but I don't see a codeable concept endpoint for these like I showed for the BodySites above.

In [11]:
# pvalues = [pv for key, pv in ccdh.EnumCRDCHCancerStageObservationObservationType.__dict__.items() if isinstance(pv, ccdh.PermissibleValue)]
# csoot_list = []
# for pv in pvalues:
#     csoot_list.append(pv.text)
# csoot_list.sort()
# csoot_list


In [12]:
pvalues = [pv for key, pv in ccdh.EnumCRDCHCancerStageObservationObservationType.__dict__.items() if isinstance(pv, ccdh.PermissibleValue)]
# pvalues
ccdh.EnumCRDCHCancerStageObservationObservationType['Pathological Metastasis (M)'].description

'M classifies the presence of distant metastasis; stage given by histopathologic examination of a surgical specimen'

In [13]:

temp = gdc_head_and_mouth[131]['diagnoses'][0]
for k,v in temp.items():
    print(k)
    print("  " + str(v))

age_at_diagnosis
  22107
ajcc_clinical_m
  M1
ajcc_clinical_n
  N1
ajcc_clinical_stage
  Stage IVC
ajcc_clinical_t
  T3
ajcc_pathologic_m
  M1
ajcc_pathologic_n
  N1
ajcc_pathologic_stage
  Stage IVC
ajcc_pathologic_t
  T3
ajcc_staging_system_edition
  7th
classification_of_tumor
  not reported
created_datetime
  None
days_to_diagnosis
  0
days_to_last_follow_up
  None
days_to_last_known_disease_status
  None
days_to_recurrence
  None
diagnosis_id
  9e30aa6c-91e6-5dd3-9512-75c162a89913
icd_10_code
  C32.9
last_known_disease_status
  not reported
morphology
  8070/3
primary_diagnosis
  Squamous cell carcinoma, NOS
prior_malignancy
  no
prior_treatment
  No
progression_or_recurrence
  not reported
site_of_resection_or_biopsy
  Larynx, NOS
state
  released
submitter_id
  TCGA-QK-A8Z8_diagnosis
synchronous_malignancy
  No
tissue_or_organ_of_origin
  Larynx, NOS
tumor_grade
  not reported
tumor_stage
  stage ivc
updated_datetime
  2019-08-08T16:48:06.888156-05:00
year_of_diagnosis
  2013


In [14]:
gdc_to_crdch_csoot_dict = [
    {"GDC": "ajcc_clinical_m", "CRDC-H": "Clinical Metastasis (M)"},
    {"GDC": "ajcc_clinical_n", "CRDC-H": "Clinical Node (N)"},
    {"GDC": "ajcc_clinical_stage", "CRDC-H": "Clinical Overall"},
    {"GDC": "ajcc_clinical_t", "CRDC-H": "Clinical Tumor (T)"},
    {"GDC": "tumor_stage", "CRDC-H": "Overall"},
    {"GDC": "ajcc_pathologic_m", "CRDC-H": "Pathological Metastasis (M)"},
    {"GDC": "ajcc_pathologic_n", "CRDC-H": "Pathological Node (N)"},
    {"GDC": "ajcc_pathologic_stage", "CRDC-H": "Pathological Overall"},
    {"GDC": "ajcc_pathologic_t", "CRDC-H": "Pathological Tumor (T)"},
    {"GDC": "age_at_diagnosis", "CRDC-H": None},
    {"GDC": "ajcc_staging_system_edition", "CRDC-H": None},
    {"GDC": "classification_of_tumor", "CRDC-H": None},
    {"GDC": "created_datetime", "CRDC-H": None},
    {"GDC": "days_to_diagnosis", "CRDC-H": None},
    {"GDC": "days_to_last_follow_up", "CRDC-H": None},
    {"GDC": "days_to_last_known_disease_status", "CRDC-H": None},
    {"GDC": "days_to_recurrence", "CRDC-H": None},
    {"GDC": "diagnosis_id", "CRDC-H": None},
    {"GDC": "icd_10_code", "CRDC-H": None},
    {"GDC": "last_known_disease_status", "CRDC-H": None},
    {"GDC": "morphology", "CRDC-H": None},
    {"GDC": "primary_diagnosis", "CRDC-H": None},
    {"GDC": "prior_malignancy", "CRDC-H": None},
    {"GDC": "prior_treatment", "CRDC-H": None},
    {"GDC": "progression_or_recurrence", "CRDC-H": None},
    {"GDC": "site_of_resection_or_biopsy", "CRDC-H": None},
    {"GDC": "state", "CRDC-H": None},
    {"GDC": "submitter_id", "CRDC-H": None},
    {"GDC": "synchronous_malignancy", "CRDC-H": None},
    {"GDC": "tissue_or_organ_of_origin", "CRDC-H": None},
    {"GDC": "tumor_grade", "CRDC-H": None},
    {"GDC": "updated_datetime", "CRDC-H": None},
    {"GDC": "year_of_diagnosis", "CRDC-H": None},
]


In [15]:
gdc_to_crdch_csoot_frame = pd.DataFrame(gdc_to_crdch_csoot_dict)

In [16]:
gdc_to_crdch_csoot_frame_complete = gdc_to_crdch_csoot_frame.loc[ ~ gdc_to_crdch_csoot_frame['CRDC-H'].isnull()]

In [17]:
gdc_to_crdch_csoot_frame_complete

,GDC,CRDC-H
0,ajcc_clinical_m,Clinical Metastasis (M)
1,ajcc_clinical_n,Clinical Node (N)
2,ajcc_clinical_stage,Clinical Overall
3,ajcc_clinical_t,Clinical Tumor (T)
4,tumor_stage,Overall
5,ajcc_pathologic_m,Pathological Metastasis (M)
6,ajcc_pathologic_n,Pathological Node (N)
7,ajcc_pathologic_stage,Pathological Overall
8,ajcc_pathologic_t,Pathological Tumor (T)


In [18]:
from linkml_runtime.dumpers import yaml_dumper

cancer_stage_method_type = None
# if diagnosis.get('ajcc_staging_system_edition') == '7th':
#     cancer_stage_method_type = 'AJCC staging system 7th edition'

# Create an observation set
obs = ccdh.CancerStageObservationSet(method_type=cancer_stage_method_type)


def diag_cc_mapper(diag_info_type, diag_info_val):
    if diag_info_val is not None:
        crdch_diag_info_type = gdc_to_crdch_csoot_frame_complete["CRDC-H"].loc[
            gdc_to_crdch_csoot_frame_complete["GDC"] == diag_info_type
        ]
        crdch_diag_info_type = list(crdch_diag_info_type)
        if len(crdch_diag_info_type) == 1:
            crdch_diag_info_type = crdch_diag_info_type[0]
            # we could look up the sescription for the mapped type enum
            #   but where would we put it in the obs structure?
            crdch_diag_info_type_desc = (
                ccdh.EnumCRDCHCancerStageObservationObservationType[
                    crdch_diag_info_type
                ].description
            )
            current_observation = ccdh.CancerStageObservation(
                observation_type=ccdh.CodeableConcept(
                    coding=[
                        ccdh.Coding(
                            code=crdch_diag_info_type,
                            system="crdch-model 1.1.5 legacy enums",
                        )
                    ],
                    text=diag_info_type,
                ),
                # valueCodeableConcept ?
                value_codeable_concept=ccdh.CodeableConcept(
                    coding=[
                        ccdh.Coding(
                            code=diag_info_val, system="crdch-model 1.1.5 legacy enums"
                        )
                    ]
                ),
            )
            obs.observations.append(current_observation)


diagnosis_dict = gdc_head_and_mouth[131]["diagnoses"][0]
for k, v in diagnosis_dict.items():
    diag_cc_mapper(k, v)


In [19]:
print(obs.observations[0])

CancerStageObservation(observation_type=CodeableConcept(coding=[Coding(code='Clinical Metastasis (M)', system='crdch-model 1.1.5 legacy enums', label=None, system_version=None, value_set=None, value_set_version=None, tag=[])], text='ajcc_clinical_m'), value_codeable_concept=CodeableConcept(coding=[Coding(code='M1', system='crdch-model 1.1.5 legacy enums', label=None, system_version=None, value_set=None, value_set_version=None, tag=[])], text=None), id=None, category=None, method_type=None, focus=None, subject=None, performed_by=None, value_entity=None)


In [20]:
print(yaml_dumper.dumps(obs))

observations:
- observation_type:
    coding:
    - code: Clinical Metastasis (M)
      system: crdch-model 1.1.5 legacy enums
    text: ajcc_clinical_m
  value_codeable_concept:
    coding:
    - code: M1
      system: crdch-model 1.1.5 legacy enums
- observation_type:
    coding:
    - code: Clinical Node (N)
      system: crdch-model 1.1.5 legacy enums
    text: ajcc_clinical_n
  value_codeable_concept:
    coding:
    - code: N1
      system: crdch-model 1.1.5 legacy enums
- observation_type:
    coding:
    - code: Clinical Overall
      system: crdch-model 1.1.5 legacy enums
    text: ajcc_clinical_stage
  value_codeable_concept:
    coding:
    - code: Stage IVC
      system: crdch-model 1.1.5 legacy enums
- observation_type:
    coding:
    - code: Clinical Tumor (T)
      system: crdch-model 1.1.5 legacy enums
    text: ajcc_clinical_t
  value_codeable_concept:
    coding:
    - code: T3
      system: crdch-model 1.1.5 legacy enums
- observation_type:
    coding:
    - code: P

## Expecting something like this:

```
observations:
- observation_type:
    text: Overall
    description: The overall stage of the disease
  valueCodeableConcept:
    text: Stage IVC
    description: Stage IVC
- observation_type:
    text: Clinical Overall
    description: The overall stage of the disease; clinical stage is determined from
      evidence acquired before treatment (including clinical examination, imaging,
      endoscopy, biopsy, surgical exploration)
  valueCodeableConcept:
    text: Stage IVC
    description: Stage IVC
- observation_type:
    text: Clinical Tumor (T)
    description: T classifies the size or direct extent of the primary tumor; clinical
      stage is determined from evidence acquired before treatment (including clinical
      examination, imaging, endoscopy, biopsy, surgical exploration)
  valueCodeableConcept:
    text: T3
    description: T3 Stage Finding
- observation_type:
    text: Clinical Node (N)
    description: N classifies the degree of spread to regional lymph nodes; clinical
      stage is determined from evidence acquired before treatment (including clinical
      examination, imaging, endoscopy, biopsy, surgical exploration)
  valueCodeableConcept:
    text: N1
    description: N1 Stage Finding
- observation_type:
    text: Clinical Metastasis (M)
    description: M classifies the presence of distant metastasis; clinical stage is
      determined from evidence acquired before treatment (including clinical examination,
      imaging, endoscopy, biopsy, surgical exploration)
  valueCodeableConcept:
    text: M1
    description: M1 Stage Finding
- observation_type:
    text: Pathological Overall
    description: The overall stage of the disease; stage given by histopathologic
      examination of a surgical specimen
  valueCodeableConcept:
    text: Stage IVC
    description: Stage IVC
- observation_type:
    text: Pathological Tumor (T)
    description: T classifies the size or direct extent of the primary tumor; stage
      given by histopathologic examination of a surgical specimen
  valueCodeableConcept:
    text: T3
    description: T3 Stage Finding
- observation_type:
    text: Pathological Node (N)
    description: N classifies the degree of spread to regional lymph nodes; stage
      given by histopathologic examination of a surgical specimen
  valueCodeableConcept:
    text: N1
    description: N1 Stage Finding
- observation_type:
    text: Pathological Metastasis (M)
    description: M classifies the presence of distant metastasis; stage given by histopathologic
      examination of a surgical specimen
  valueCodeableConcept:
    text: M1
    description: M1 Stage Finding

```

In [21]:
def create_stage_observation(type, value):
    """ Create a CCDHCancerStageObservation from a type of observation and a codeable concept."""
    # As with the body site example above, we need to map GDC values into the values
    # allowed under the CCDH model.
    stage_mappings = {
        'not reported': 'Not Reported',
        'unknown': 'Unknown',
        'stage i': 'Stage I',
        'stage ii': 'Stage II',
        'stage iii': 'Stage III',
        'stage iva': 'Stage IVA',
        'stage ivb': 'Stage IVB',
        'stage ivc': 'Stage IVC',
    }
    
    if value in stage_mappings:
        return ccdh.CancerStageObservation(
            observation_type=type,
            valueCodeableConcept=stage_mappings[value]
        )
    
    return ccdh.CancerStageObservation(
        observation_type=type,
        valueCodeableConcept=value
    )

def create_stage_from_gdc(diagnosis):
    cancer_stage_method_type = None
    if diagnosis.get('ajcc_staging_system_edition') == '7th':
        cancer_stage_method_type = 'AJCC staging system 7th edition'

    # Create an observation set 
    obs = ccdh.CancerStageObservationSet(
        method_type=cancer_stage_method_type
    )
    
    # Add observations for every type of observation in the GDC diagnosis.
    if diagnosis.get('tumor_stage') is not None:
        obs.observations.append(create_stage_observation('Overall', diagnosis.get('tumor_stage')))
        
    if diagnosis.get('ajcc_clinical_stage') is not None:
        obs.observations.append(create_stage_observation('Clinical Overall', diagnosis.get('ajcc_clinical_stage')))
        
    if diagnosis.get('ajcc_clinical_t') is not None:
        obs.observations.append(create_stage_observation('Clinical Tumor (T)', diagnosis.get('ajcc_clinical_t')))
        
    if diagnosis.get('ajcc_clinical_n') is not None:
        obs.observations.append(create_stage_observation('Clinical Node (N)', diagnosis.get('ajcc_clinical_n')))
        
    if diagnosis.get('ajcc_clinical_m') is not None:
        obs.observations.append(create_stage_observation('Clinical Metastasis (M)', diagnosis.get('ajcc_clinical_m')))
    
    if diagnosis.get('ajcc_pathologic_stage') is not None:
        obs.observations.append(create_stage_observation('Pathological Overall', diagnosis.get('ajcc_pathologic_stage')))
        
    if diagnosis.get('ajcc_pathologic_t') is not None:
        obs.observations.append(create_stage_observation('Pathological Tumor (T)', diagnosis.get('ajcc_pathologic_t')))
        
    if diagnosis.get('ajcc_pathologic_n') is not None:
        obs.observations.append(create_stage_observation('Pathological Node (N)', diagnosis.get('ajcc_pathologic_n')))
        
    if diagnosis.get('ajcc_pathologic_m') is not None:
        obs.observations.append(create_stage_observation('Pathological Metastasis (M)', diagnosis.get('ajcc_pathologic_m')))
    
    return obs

# Test transform with the diagnosis from the first loaded case.
# Note that the resulting CancerStageObservationSet contains descriptions for the concepts included in it.
# example_observation_set = create_stage_from_gdc(gdc_head_and_mouth[131]['diagnoses'][0], ccdh.Subject(id='1234'))
example_observation_set = create_stage_from_gdc(gdc_head_and_mouth[131]['diagnoses'][0])
example_observation_set

TypeError: type object argument after ** must be a mapping, not str

Reading Python Data Classes in its default text output can be difficult! However, we can use LinkML's [YAML](https://en.wikipedia.org/wiki/YAML) dumper to display this Cancer Stage Observation Set as a YAML string. YAML objects are a good way to export LinkML data, and include detailed descriptions of all the enumerations referenced from this object. We currently include basic descriptions for the permissible values (see e.g. "N1 Stage Finding" below), but we will include more detailed descriptions in the future.

In [ ]:
from linkml_runtime.dumpers import yaml_dumper

print(yaml_dumper.dumps(example_observation_set))

## Trying to replicate up to this ^ point today

Diagnoses can contain samples, which we transform into [CCDH Samples](https://cancerdhc.github.io/ccdhmodel/latest/Specimen/).

In [ ]:
def transform_sample_to_specimen(sample):
    """
    A method for transforming a GDC Sample into CCDH Specimen.
    """
    specimen = ccdh.Specimen(id = sample.get('sample_id'))
    specimen.source_material_type = sample.get('sample_type')
    specimen.general_tissue_morphology = sample.get('tissue_type')
    specimen.specific_tissue_morphology = sample.get('tumor_code')
    specimen.tumor_status_at_collection = sample.get('tumor_descriptor')
    specimen.creation_activity = ccdh.SpecimenCreationActivity(
        date_ended=ccdh.TimePoint(
            dateTime=sample.get('created_datetime')
        )
    )
    return specimen

# Let's try creating a test specimen.
test_specimen = transform_sample_to_specimen(gdc_head_and_mouth[2]['samples'][0])
test_specimen

We can now transform an entire diagnosis into a [CCDH Diagnosis](https://cancerdhc.github.io/ccdhmodel/latest/Diagnosis/).

In [ ]:
def transform_diagnosis(diagnosis, case):
    ccdh_diagnosis = ccdh.Diagnosis(
        id=diagnosis.get('diagnosis_id'),
        condition=diagnosis.get('primary_diagnosis'),
        morphology=diagnosis.get('morphology'),
        grade=diagnosis.get('grade'),
        stage=create_stage_from_gdc(diagnosis),
        year_at_diagnosis=diagnosis.get('year_of_diagnosis'),
        related_specimen=[
            transform_sample_to_specimen(
                sample
            ) for sample in case.get('samples')
        ]
    )
    ccdh_diagnosis.identifier = [
        ccdh.Identifier(
            system='GDC-submitter-id',
            value=diagnosis.get('submitter_id')
        )
    ]
    
    if 'primary_site' in case and case['primary_site'] != '':
        body_site = create_body_site(case['primary_site'])
        if body_site is not None:
            ccdh_diagnosis.metastatic_site.append(body_site)

    return ccdh_diagnosis

example_diagnosis = transform_diagnosis(gdc_head_and_mouth[131]['diagnoses'][0], gdc_head_and_mouth[131])
print(yaml_dumper.dumps(example_diagnosis))

## Exporting Python Data Classes as JSON-LD

Python Data Classes can be exported as [JSON-LD](https://en.wikipedia.org/wiki/JSON-LD), allowing CCDH instance data to be shared in a [JSON](https://en.wikipedia.org/wiki/JSON)-based [RDF](https://en.wikipedia.org/wiki/Resource_Description_Framework) format. RDF formats are particularly useful in sharing data, since they allow us to share [Linked Data](https://en.wikipedia.org/wiki/Linked_data) that can be understood by other consumers.

In [ ]:
from linkml.generators.jsonldcontextgen import ContextGenerator
from linkml_runtime.dumpers import json_dumper

jsonldContext = ContextGenerator('ccdh/ccdhmodel.yaml').serialize()
jsonldContextAsDict = json.loads(jsonldContext)

# Display the example diagnosis we constructed in a previous step.
print(json_dumper.dumps(example_diagnosis, jsonldContext))

We can also transform all the diagnoses in this file and store them in a file as JSON-LD.

In [ ]:
diagnoses = []
for case in gdc_head_and_mouth:
    for diagnosis in case['diagnoses']:
        diagnoses.append(transform_diagnosis(diagnosis, case))

jsonld = json_dumper.dumps({
    '@graph': diagnoses,
    '@context': jsonldContextAsDict
})
with open('head-and-mouth/diagnoses.jsonld', 'w') as file:
    file.write(jsonld)

## Converting JSON-LD to Turtle

While JSON-LD is a full dialect of RDF, people are more familiar looking at RDF in a format like [Turtle](https://en.wikipedia.org/wiki/Turtle_(syntax)). We can convert the generated JSON-LD output into Turtle by using the [rdflib](https://rdflib.readthedocs.io/en/stable/) package.

Note that this section is intended to be illustrative -- these are *not* finalized IRIs for properties and entities. We will choose IRIs and develop a canonical RDF representation in future phases of development.

In [ ]:
# We can read this JSON-LD in Turtle.
from rdflib import Graph

g = Graph()
g.parse(data=jsonld, format="json-ld")
rdfAsTurtle = g.serialize(format="turtle").decode()
print(''.join(rdfAsTurtle[0:1000]))
with open('head-and-mouth/diagnoses.ttl', 'w') as file:
    file.write(rdfAsTurtle)

---

In [ ]:
# maybe not that helpful
# ccdh.BodySite.__dict__
# see also https://cancerdhc.github.io/ccdhmodel/v1.1/BodySite/
# takes a CodeableConcept in the mandatory site and optional qualifier 0 slots
# CodeableConcept takes the following slots
# 
#     coding 0..*
#         Description: A reference to a code defined by a terminology system
#         Range: Coding
#     text 0..1
#         Description: A human language representation of the concept represented by the Coding.
#         Range: CrdchString


In [ ]:
uberon_larynx_coding = ccdh.Coding(code="UBERON:0001737", system="http://purl.obolibrary.org/obo/uberon.owl")
uberon_larynx_coding

In [ ]:
uberon_larynx__cc = ccdh.CodeableConcept(coding=[test_coding])
uberon_larynx__cc

In [ ]:
uberon_larynx_bs = ccdh.BodySite(site=test_cc)
uberon_larynx_bs

In [ ]:
ccdh.CodeableConcept(
    coding=[
        ccdh.Coding(
            code="UBERON:0001737",
            system="http://purl.obolibrary.org/obo/uberon.owl",
            label="larynx",
        )
    ]
)
